# Transfer Learning

### Aufgabe
_Wählen Sie nun 5 andere Hunderassen aus. Nutzen Sie Transfer Learning zur Klassifikation dieser
Rassen, d.h. laden Sie die Gewichte aus Ihrem in Teilaufgabe B5 trainierten Modell als initiale
Gewichte für das neue Training. Beschreiben Sie ihr Vorgehen im Kurzreport und setzen Sie die
Evaluation dieses erneuten Trainings in Relation zu den Evaluationsergebnissen ihres ursprünglichen
Modells._

In [6]:
import keras
import tensorflow as tf
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import sys
import os

%matplotlib inline

notebook_dir = Path().resolve()
sys.path.append(str(notebook_dir.parent))

from utils.setup import setup_dataset, generateDatasetArrays

# load configuration
import config as Config

# config overrides
Config.DOG_LABEL_IDS = [5, 12, 45, 86, 100] # 5 andere Hunderassen

In [7]:
train_dataset, test_dataset, label_lookup_table, info = setup_dataset('../dataset', labels=Config.DOG_LABEL_IDS)

def preprocess(data):
    img = data["image"]
    data["original_image"] = tf.identity(data["image"])
    img = tf.image.resize(img, Config.RESIZE_SIZE)
    data["image"] = img
    data["label"] = label_lookup_table.lookup(data["label"])
    return data

train_dataset = (
    train_dataset
        .map(preprocess)
        .shuffle(1000)
        .prefetch(tf.data.AUTOTUNE)
)

test_dataset = (
    test_dataset
        .map(preprocess)
        .shuffle(100)
        .prefetch(tf.data.AUTOTUNE)
)

In [8]:
model = keras.models.load_model(os.path.join(Config.MODEL_FOLDER, Config.MODEL_FILENAME))

model.add(keras.layers.Dense(256, activation='relu'))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(Config.NUM_CLASSES, activation='softmax'))

# Freeze the layers except the last three
for layer in model.layers[:-3]:
    layer.trainable = False

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=Config.LEARNING_RATE),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

2024-12-13 22:19:27.829133: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 117964800 exceeds 10% of free system memory.
2024-12-13 22:19:27.915383: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 117964800 exceeds 10% of free system memory.


In [ ]:
def visualize_history(history):

    # Loss-Werte
    loss = history['loss']
    val_loss = history['val_loss']

    # Accuracy-Werte (falls verwendet)
    accuracy = history.get('accuracy')
    val_accuracy = history.get('val_accuracy')

    # Epochen erstellen
    epochs = range(1, len(loss) + 1)

    # Plot für den Loss
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(epochs, loss, 'bo-', label='Trainings Loss')
    plt.plot(epochs, val_loss, 'ro-', label='Validierungs Loss')
    plt.title('Trainings und Validierungs Loss')
    plt.xlabel('Epochen')
    plt.ylabel('Loss')
    plt.legend()

    # Plot für die Accuracy
    plt.subplot(1, 2, 2)
    plt.plot(epochs, accuracy, 'bo-', label='Trainings Genauigkeit')
    plt.plot(epochs, val_accuracy, 'ro-', label='Validierungs Genauigkeit')
    plt.title('Trainings und Validierungs Genauigkeit')
    plt.xlabel('Epochen')
    plt.ylabel('Genauigkeit')
    plt.legend()

    plt.tight_layout()
    plt.show()

In [9]:
images, labels = generateDatasetArrays(train_dataset)
images_test, labels_test = generateDatasetArrays(test_dataset)

history = model.fit(
    images, labels,
    validation_data=(images_test, labels_test),
    epochs=Config.EPOCHS,
    batch_size=Config.BATCH_SIZE,
    verbose=1 if Config.DEBUG else 0
)

# Feinabstimmung des gesamten Modells
for layer in model.layers:
    layer.trainable = True  # Alle Schichten freigeben

# Reduzierte Lernrate für Fine-Tuning
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.0001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)
print("Finetuning des gesamten Modells")
history_finetune = model.fit(
    images, labels,
    validation_data=(images_test, labels_test),
    epochs=5,
    batch_size=16,
    verbose=1 if Config.DEBUG else 0
)

# Kombinieren der Trainingsergebnisse, damit das Trainieren der letzten Ebenenen, sowie das Feintuning in einem Graph visualisiert wird
for key in history.history.keys():
    history.history[key].extend(history_finetune.history[key])

visualize_history(history.history)

2024-12-13 22:19:28.128582: I tensorflow/core/kernels/data/tf_record_dataset_op.cc:376] The default buffer size is 262144, which is overridden by the user specified `buffer_size` of 8388608
2024-12-13 22:19:28.389189: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


NameError: name 'new_model' is not defined

In [ ]:
# Vorhersagen für Testdaten
y_pred_new = new_model.predict(X_test_new)

# Konvertiere die Vorhersagen zu Klassenindizes
y_pred_classes_new = np.argmax(y_pred_new, axis=1)
y_test_int_new = np.argmax(y_test_new, axis=1)

# Konfusionsmatrix
conf_matrix_new = confusion_matrix(y_test_int_new, y_pred_classes_new)
disp_new = ConfusionMatrixDisplay(confusion_matrix=conf_matrix_new, display_labels=le_new.classes_)
disp_new.plot(cmap=plt.cm.Blues, xticks_rotation=90)
plt.title("Konfusionsmatrix - Transfer Learning")

#y_pred_classes_new = to_categorical(y_pred_classes_new)

#y_pred_classes_new = to_categorical(y_pred_classes_new)
report = classification_report(y_test_int_new, y_pred_classes_new, target_names=le_new.classes_)

print(report)


#plt.show()